## Means, TACS, and metadata
Notebook used to create the following pickle data files with readouts from the combined hedyPET train+validation set (n=80):
 - `readouts/metadata.csv` Participant metadata (weight, demographic-group, and more)- `readouts/acdynPSF_tacs_80.pkl.gz` PET TACs for all segmented regions
 - `readouts/acstatPSF_mean_80.pkl.gz` PET static means for all segmented regions
 - `readouts/patlak_ki_80.pkl.gz` Patlak Ki values for all segmented regions for different input functions (IFs) and number off rames

Extensive metadata and original image files available at X

In [2]:
from hedypet.utils import STATIC_ROOT, load_splits, get_time_frames_midpoint, DYNAMIC_ROOT
from hedypet.utils import get_participant_metadata, get_norm_consts
from nifti_dynamic.tacs import load_tac
from nifti_dynamic.patlak import roi_patlak
from parse import parse
from tqdm import tqdm
import pandas as pd 
import os 
import json
import warnings

## Helper functions

In [3]:
def load_tsv(file_path):
    df = pd.read_csv(file_path,sep="\t")
    return {k:v for k,v in zip(list(df["index"]),list(df.name))}

region_names = {
    'ts_total' :load_tsv(next((STATIC_ROOT / "derivatives/totalsegmentator").glob("**/*total*.tsv"))),
    'ts_body' : load_tsv(next((STATIC_ROOT / "derivatives/totalsegmentator").glob("**/*body*.tsv"))),
    'ts_tissue' :load_tsv(next((STATIC_ROOT / "derivatives/totalsegmentator").glob("**/*tissue*.tsv"))),
    'synthsegparc' : load_tsv(next((STATIC_ROOT / "derivatives/synthseg").glob("**/*synthseg*.tsv"))),
    'synthseg' : load_tsv(next((STATIC_ROOT / "derivatives/synthseg").glob("**/*synthseg*.tsv"))),
    'totalimage' : {1:"body"},
}

region_names_aorta = {1:"aorta_ascending",
2:"aorta_top",
3:"aorta_descending_upper",
4:"aorta_descending_lower"}

def task_and_ix_to_region_name(task,ix):
    ix = int(ix)
    if task.startswith("aorta"):
        return region_names_aorta[ix]
    else:
        return region_names[task][ix]

subs = load_splits()["train0"]+load_splits()["val0"]

## Metadata
Extraction of metadata for each participant

In [3]:
if not os.path.exists(df_path := "../../../readouts/metadata_80.csv"):
    data = []
    for sub in load_splits()["train0"]+load_splits()["val0"]:
        x = get_participant_metadata(sub)
        x["SUV Denominator [Bq/mL]"] = get_norm_consts(sub)["suv"]
        x["SUL Decazes Denominator [Bq/mL]"] = get_norm_consts(sub)["sul_decazes"]
        x["SUL Janma Denominator [Bq/mL]"] = get_norm_consts(sub)["sul_janma"]
        x["SUL James Denominator [Bq/mL]"] = get_norm_consts(sub)["sul_james"]
        data.append(x)

    df = pd.DataFrame(data)
    df = df.rename({"InjectedRadioactivity":"Injected Activity [MBq]","participant_id":"Subject","weight":"Weight [kg]","demographic-group":"Demographic Group [Sex,Age]"},axis=1)
    df = df.drop(["age","height","sex","blanket"],axis=1)
    df.to_csv(df_path,index=False)
else:
    df = pd.read_csv(df_path)

print("="*10 + "[Metadata Dataframe]" + "="*10,end="\n\n")
print("Columns:", list(df.columns),end="\n\n")
print("Subjects:", df["Subject"].nunique(),end="\n\n")
print(df["Demographic Group [Sex,Age]"].value_counts(),end="\n\n")

==========[Metadata Dataframe]==========

Columns: ['Subject', 'Weight [kg]', 'Demographic Group [Sex,Age]', 'suv_const', 'sul_janma_const', 'sul_james_const', 'Injected Activity [MBq]', 'SUV Denominator [Bq/mL]', 'SUL Decazes Denominator [Bq/mL]', 'SUL Janma Denominator [Bq/mL]', 'SUL James Denominator [Bq/mL]']

Subjects: 80

Demographic Group [Sex,Age]
F18-34    10
F35-49    10
F50-69    10
F70-99    10
M18-34    10
M35-49    10
M50-69    10
M70-99    10
Name: count, dtype: int64



## Time Activity Curves
Combines the time activity curves from the acdynPSF dynamic PET into a single dataframe

In [6]:
df_path = "../../../readouts/tacs_80.pkl.gz"
df_path_input_function = "../../../readouts/input_function_tacs_80.pkl.gz"

if not os.path.exists(df_path) or not os.path.exists(df_path_input_function):
    data = []
    
    for sub in tqdm(subs):
        tacs_root = (DYNAMIC_ROOT / f"derivatives/tacs/{sub}/acdynPSF")
        tacs = list(tacs_root.glob("**/tac*"))


        # Load the TAC for each ROI with/without erosion
        for tac_roi_path in tacs:

            frame_time_start, frame_duration,mu_organ, std_organ, n_organ = load_tac(tac_roi_path)
            if (frame_duration==0).all():
                frame_duration = np.array(list(frame_time_start[1:]) + [4200]) - frame_time_start

            frame_time_middle = frame_time_start+frame_duration/2
            # Save data to dataframe
            frame_ixs = list(range(len(mu_organ)))            
            tags = parse('{}/tacs/{sub}/acdynPSF/{task}/erosion-{erosion}/tac_{ix}.csv',str(tac_roi_path)).named
            vals = {"PET Mean [Bq/mL]":mu_organ,"PET STD [Bq/mL]":std_organ,"Voxel Count":n_organ,"Frame Index":frame_ixs, "Frame Time Middle [s]":frame_time_middle}
            vals.update(tags)
            vals["Label Name"] = task_and_ix_to_region_name(vals["task"],vals["ix"])
            data.append(pd.DataFrame(vals))

    # Rename and save 
    df = pd.concat(data)
    df = df.rename({"sub":"Subject","task":"Task","ix":"Label Index","erosion":"Erosion Iterations"},axis="columns")
    df["Volume [mL]"] = df["Voxel Count"]*(1.65*1.65* 1.65) / 1000
    df["Erosion Iterations"] = df["Erosion Iterations"].astype(int)
    df = df[['Subject', 'Task','Label Index', 'Label Name', 'Erosion Iterations', 'Frame Index', 'Frame Time Middle [s]', 'PET Mean [Bq/mL]', 'PET STD [Bq/mL]', 'Voxel Count',  'Volume [mL]']]
    
    df_input_functions = df[df.Task.str.startswith("aortavois")]
    df_organs = df[~df.Task.str.startswith("aortavois")]
    df_input_functions.to_pickle(df_path_input_function)
    df_organs.to_pickle(df_path)
else:
    df_input_functions = pd.read_pickle(df_path_input_function)
    df_organs = pd.read_pickle(df_path)

print("="*10 + "[Time Activity Curves Dataframe]" + "="*10,end="\n\n")
print("Columns:", list(df.columns),end="\n\n")
print("Segmentation tasks:", list(df["Task"].unique()),end="\n\n")
print("N Unique regions:", df["Label Name"].nunique(), end="\n\n")
print("Unique erosions:", df["Erosion Iterations"].unique(),end="\n\n")
print("Rows:",len(df),end="\n\n\n")



print("="*10 + "[Input Functions Time Activity Curves Dataframe]" + "="*10,end="\n\n")
print("Columns:", list(df_input_functions.columns),end="\n\n")
print("Segmentation tasks:", list(df_input_functions["Task"].unique()),end="\n\n")
print("N Unique regions:", df_input_functions["Label Name"].nunique(), end="\n\n")
print("Unique erosions:", df_input_functions["Erosion Iterations"].unique(),end="\n\n")
print("Rows:",len(df_input_functions))

100%|██████████| 80/80 [01:16<00:00,  1.05it/s]


==========[Time Activity Curves Dataframe]==========

Columns: ['Subject', 'Task', 'Label Index', 'Label Name', 'Erosion Iterations', 'Frame Index', 'Frame Time Middle [s]', 'PET Mean [Bq/mL]', 'PET STD [Bq/mL]', 'Voxel Count', 'Volume [mL]']

Segmentation tasks: ['ts_total', 'synthseg', 'synthsegparc', 'ts_tissue', 'ts_body', 'aortasegments', 'aortavois_ml-full_width-3', 'aortavois_ml-1_width-3', 'aortavois_ml-2_width-5', 'totalimage']

N Unique regions: 227

Unique erosions: [0 1]

Rows: 2860533


==========[Input Functions Time Activity Curves Dataframe]==========

Columns: ['Subject', 'Task', 'Label Index', 'Label Name', 'Erosion Iterations', 'Frame Index', 'Frame Time Middle [s]', 'PET Mean [Bq/mL]', 'PET STD [Bq/mL]', 'Voxel Count', 'Volume [mL]']

Segmentation tasks: ['aortavois_ml-full_width-3', 'aortavois_ml-1_width-3', 'aortavois_ml-2_width-5']

N Unique regions: 4

Unique erosions: [0]

Rows: 66240


## Static Organ Means
Combines the static organ means from the acstatPSF reconstruction into a single dataframe

In [15]:
if not os.path.exists(df_path := "../../../readouts/means_80.pkl.gz"):
    data = []
    
    for sub in tqdm(subs):
        tacs_root = (STATIC_ROOT / f"derivatives/tacs/{sub}/acstatPSF")

        tacs = list(tacs_root.glob("**/tac*"))
                
        # Load the ROI means for all regions (with/without eerosion)
        for tac_roi_path in tacs:
            _,_,mu_organ, std_organ, n_organ = load_tac(tac_roi_path)

            # Save data to dataframe
            tags = parse('{}/tacs/{sub}/acstatPSF/{task}/erosion-{erosion}/tac_{ix}.csv',str(tac_roi_path)).named
            vals = {"PET Mean [Bq/mL]":float(mu_organ.item()),"PET STD [Bq/mL]":float(std_organ.item()),"Voxel Count":int(n_organ.item())}
            vals.update(tags)
            vals["Label Name"] = task_and_ix_to_region_name(vals["task"],vals["ix"])
            data.append(vals)

    # Rename and save 
    df = pd.DataFrame(data)
    df = df.rename({"sub":"Subject","task":"Task","ix":"Label Index","erosion":"Erosion Iterations"},axis="columns")
    df["Volume [mL]"] = df["Voxel Count"]*(1.65*1.65* 2.0) / 1000
    df["Erosion Iterations"] = df["Erosion Iterations"].astype(int)
    df = df[['Subject', 'Task','Label Index', 'Label Name', 'Erosion Iterations', 'PET Mean [Bq/mL]', 'PET STD [Bq/mL]', 'Voxel Count', 'Volume [mL]']]
    df.to_pickle(df_path)
    
else:
    df = pd.read_pickle(df_path)

print("="*10 + "[Means Dataframe]" + "="*10,end="\n\n")
print("Columns:", list(df.columns),end="\n\n")
print("Segmentation tasks:", list(df["Task"].unique()),end="\n\n")
print("N Unique regions:", df["Label Name"].nunique(), end="\n\n")
print("Unique erosions:", df["Erosion Iterations"].unique(),end="\n\n")
print("Rows:",len(df))


100%|██████████| 80/80 [02:04<00:00,  1.55s/it]


==========[Means Dataframe]==========

Columns: ['Subject', 'Task', 'Label Index', 'Label Name', 'Erosion Iterations', 'PET Mean [Bq/mL]', 'PET STD [Bq/mL]', 'Voxel Count', 'Volume [mL]']

Segmentation tasks: ['ts_total', 'synthseg', 'synthsegparc', 'ts_tissue', 'ts_body', 'totalimage']

N Unique regions: 223

Unique erosions: [0 1]

Rows: 39809


## Patlak

Computes and saves the Patlak Ki for different organ and input-function combinations

In [38]:
frame_duration

array([ 0.00e+00, -2.00e+00, -4.00e+00, -6.00e+00, -8.00e+00, -1.00e+01,
       -1.20e+01, -1.40e+01, -1.60e+01, -1.80e+01, -2.00e+01, -2.20e+01,
       -2.40e+01, -2.60e+01, -2.80e+01, -3.00e+01, -3.20e+01, -3.40e+01,
       -3.60e+01, -3.80e+01, -4.00e+01, -4.50e+01, -5.00e+01, -5.50e+01,
       -6.00e+01, -6.50e+01, -7.00e+01, -7.50e+01, -8.00e+01, -8.50e+01,
       -9.00e+01, -1.00e+02, -1.10e+02, -1.20e+02, -1.30e+02, -1.40e+02,
       -1.50e+02, -1.60e+02, -1.70e+02, -1.80e+02, -1.90e+02, -2.00e+02,
       -2.10e+02, -2.20e+02, -2.30e+02, -2.40e+02, -3.00e+02, -3.60e+02,
       -4.20e+02, -4.80e+02, -5.40e+02, -6.00e+02, -7.20e+02, -8.40e+02,
       -9.60e+02, -1.08e+03, -1.20e+03, -1.32e+03, -1.44e+03, -1.56e+03,
       -1.68e+03, -1.80e+03, -2.10e+03, -2.40e+03, -2.70e+03, -3.00e+03,
       -3.30e+03, -3.60e+03, -3.60e+03])

In [40]:
frame_duration

array([ 2.0e+00,  2.0e+00,  2.0e+00,  2.0e+00,  2.0e+00,  2.0e+00,
        2.0e+00,  2.0e+00,  2.0e+00,  2.0e+00,  2.0e+00,  2.0e+00,
        2.0e+00,  2.0e+00,  2.0e+00,  2.0e+00,  2.0e+00,  2.0e+00,
        2.0e+00,  2.0e+00,  5.0e+00,  5.0e+00,  5.0e+00,  5.0e+00,
        5.0e+00,  5.0e+00,  5.0e+00,  5.0e+00,  5.0e+00,  5.0e+00,
        1.0e+01,  1.0e+01,  1.0e+01,  1.0e+01,  1.0e+01,  1.0e+01,
        1.0e+01,  1.0e+01,  1.0e+01,  1.0e+01,  1.0e+01,  1.0e+01,
        1.0e+01,  1.0e+01,  1.0e+01,  6.0e+01,  6.0e+01,  6.0e+01,
        6.0e+01,  6.0e+01,  6.0e+01,  1.2e+02,  1.2e+02,  1.2e+02,
        1.2e+02,  1.2e+02,  1.2e+02,  1.2e+02,  1.2e+02,  1.2e+02,
        1.2e+02,  3.0e+02,  3.0e+02,  3.0e+02,  3.0e+02,  3.0e+02,
        3.0e+02,  3.0e+02, -3.6e+03])

In [41]:
np.array(list(frame_time_start[1:]) + [300]) 

array([2.00e+00, 4.00e+00, 6.00e+00, 8.00e+00, 1.00e+01, 1.20e+01,
       1.40e+01, 1.60e+01, 1.80e+01, 2.00e+01, 2.20e+01, 2.40e+01,
       2.60e+01, 2.80e+01, 3.00e+01, 3.20e+01, 3.40e+01, 3.60e+01,
       3.80e+01, 4.00e+01, 4.50e+01, 5.00e+01, 5.50e+01, 6.00e+01,
       6.50e+01, 7.00e+01, 7.50e+01, 8.00e+01, 8.50e+01, 9.00e+01,
       1.00e+02, 1.10e+02, 1.20e+02, 1.30e+02, 1.40e+02, 1.50e+02,
       1.60e+02, 1.70e+02, 1.80e+02, 1.90e+02, 2.00e+02, 2.10e+02,
       2.20e+02, 2.30e+02, 2.40e+02, 3.00e+02, 3.60e+02, 4.20e+02,
       4.80e+02, 5.40e+02, 6.00e+02, 7.20e+02, 8.40e+02, 9.60e+02,
       1.08e+03, 1.20e+03, 1.32e+03, 1.44e+03, 1.56e+03, 1.68e+03,
       1.80e+03, 2.10e+03, 2.40e+03, 2.70e+03, 3.00e+03, 3.30e+03,
       3.60e+03, 3.90e+03, 3.00e+02])

In [4]:
warnings.filterwarnings("ignore")
import numpy as np 

if not os.path.exists(df_path := "../../../readouts/patlak_ki_80.pkl.gz"):

    #Number of last frames to perform Patlak regression on
    frames = [2,3,4,5,6,7,8]
    ki_data = []

    for sub in tqdm(subs):

        #Find all TACs (organs, input functions, and with/without erosion)
        tacs_root = (DYNAMIC_ROOT / f"derivatives/tacs/{sub}/acdynPSF")
        tacs = list(tacs_root.glob("**/tac*"))

        #Divide into input function TACs and ROI tacs
        tacs_if = [x for x in tacs if "aortavois" in str(x)]
        tacs_roi = [x for x in tacs if "aorta" not in str(x)]
        
        # For each input function
        for tac_if_path in tacs_if:
            frame_time_start, frame_duration,tac_if, _, _ = load_tac(tac_if_path)
            if (frame_duration==0).all():
                frame_duration = np.array(list(frame_time_start[1:]) + [4200]) - frame_time_start

            t_middle = frame_time_start+frame_duration/2
            # For each organ
            for tac_roi_path in tacs_roi:
                _,_, tac_organ, _, n = load_tac(tac_roi_path)
            
                # For different number of regression Frames
                for frame in frames:

                    # Run Patlak analysis
                    slope, intercept, X, Y = roi_patlak(tac_organ,tac_if,t_middle,frame)

                    # And parse the data for the dataframe
                    tags_if = parse('{}/tacs/{}/acdynPSF/{task}/erosion-{erosion}/tac_{ix}.csv',str(tac_if_path)).named
                    tags_organ = parse('{}/tacs/{sub}/acdynPSF/{task}/erosion-{erosion}/tac_{ix}.csv',str(tac_roi_path)).named
                    if_tag = tags_if["task"]+"_"+task_and_ix_to_region_name(tags_if["task"], tags_if["ix"])
                    series = {"Patlak Ki":float(slope),"Voxel Count":int(n[0]),"Regression Frames":frame}
                    series["Input Function Identifier"] = if_tag
                    series.update(tags_organ)
                    series["Label Name"] = task_and_ix_to_region_name(series["task"], series["ix"])
                    
                    ki_data.append(series)
                    
    # Rename and save 
    df = pd.DataFrame(ki_data)
    df = df.rename({"sub":"Subject","task":"Task","ix":"Label Index","erosion":"Erosion Iterations"},axis="columns")
    df["Volume [mL]"] = df["Voxel Count"]*(1.65*1.65* 1.65) / 1000
    df["Erosion Iterations"] = df["Erosion Iterations"].astype(int)
    df = df[['Subject','Task',  'Label Index', 'Label Name', 'Erosion Iterations', 'Input Function Identifier', 'Regression Frames', 'Patlak Ki', 'Voxel Count', 'Volume [mL]']]
    df.to_pickle(df_path)
else:
    df = pd.read_pickle(df_path)

print("="*10 + "[Patlak Dataframe]" + "="*10,end="\n\n")
print("Columns:", list(df.columns),end="\n\n")
print("Segmentation tasks:", list(df["Task"].unique()),end="\n\n")
print("N Unique regions:", df["Label Name"].nunique(), end="\n\n")
print("Unique erosions:", df["Erosion Iterations"].unique(),end="\n\n")
print("Unique patlak frames:", df["Regression Frames"].unique(),end="\n\n")
print("Unique input functions" , df["Input Function Identifier"].unique(),end="\n\n")
print("Rows:",len(df))

100%|██████████| 80/80 [35:11<00:00, 26.40s/it]


==========[Patlak Dataframe]==========

Columns: ['Subject', 'Task', 'Label Index', 'Label Name', 'Erosion Iterations', 'Input Function Identifier', 'Regression Frames', 'Patlak Ki', 'Voxel Count', 'Volume [mL]']

Segmentation tasks: ['ts_total', 'synthseg', 'synthsegparc', 'ts_tissue', 'ts_body', 'totalimage']

N Unique regions: 223

Unique erosions: [0 1]

Unique patlak frames: [2 3 4 5 6 7 8]

Unique input functions ['aortavois_ml-full_width-3_aorta_ascending'
 'aortavois_ml-full_width-3_aorta_top'
 'aortavois_ml-full_width-3_aorta_descending_upper'
 'aortavois_ml-full_width-3_aorta_descending_lower'
 'aortavois_ml-1_width-3_aorta_ascending'
 'aortavois_ml-1_width-3_aorta_top'
 'aortavois_ml-1_width-3_aorta_descending_upper'
 'aortavois_ml-1_width-3_aorta_descending_lower'
 'aortavois_ml-2_width-5_aorta_ascending'
 'aortavois_ml-2_width-5_aorta_top'
 'aortavois_ml-2_width-5_aorta_descending_upper'
 'aortavois_ml-2_width-5_aorta_descending_lower']

Rows: 3347988


In [32]:
frame_time_start, frame_time_end


(array([0.00e+00, 2.00e+00, 4.00e+00, 6.00e+00, 8.00e+00, 1.00e+01,
        1.20e+01, 1.40e+01, 1.60e+01, 1.80e+01, 2.00e+01, 2.20e+01,
        2.40e+01, 2.60e+01, 2.80e+01, 3.00e+01, 3.20e+01, 3.40e+01,
        3.60e+01, 3.80e+01, 4.00e+01, 4.50e+01, 5.00e+01, 5.50e+01,
        6.00e+01, 6.50e+01, 7.00e+01, 7.50e+01, 8.00e+01, 8.50e+01,
        9.00e+01, 1.00e+02, 1.10e+02, 1.20e+02, 1.30e+02, 1.40e+02,
        1.50e+02, 1.60e+02, 1.70e+02, 1.80e+02, 1.90e+02, 2.00e+02,
        2.10e+02, 2.20e+02, 2.30e+02, 2.40e+02, 3.00e+02, 3.60e+02,
        4.20e+02, 4.80e+02, 5.40e+02, 6.00e+02, 7.20e+02, 8.40e+02,
        9.60e+02, 1.08e+03, 1.20e+03, 1.32e+03, 1.44e+03, 1.56e+03,
        1.68e+03, 1.80e+03, 2.10e+03, 2.40e+03, 2.70e+03, 3.00e+03,
        3.30e+03, 3.60e+03, 3.90e+03]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [29]:
df

,Subject,Task,Label Index,Label Name,Erosion Iterations,Input Function Identifier,Regression Frames,Patlak Ki,Voxel Count,Volume [mL]
0,sub-072,ts_total,1,spleen,0,aortavois_ml-full_width-3_aorta_ascending,2,-0.001252,44072,197.976933
1,sub-072,ts_total,1,spleen,0,aortavois_ml-full_width-3_aorta_ascending,3,0.007039,44072,197.976933
2,sub-072,ts_total,1,spleen,0,aortavois_ml-full_width-3_aorta_ascending,4,0.004963,44072,197.976933
3,sub-072,ts_total,1,spleen,0,aortavois_ml-full_width-3_aorta_ascending,5,0.005037,44072,197.976933
4,sub-072,ts_total,1,spleen,0,aortavois_ml-full_width-3_aorta_ascending,6,0.004364,44072,197.976933
...,...,...,...,...,...,...,...,...,...,...
3347983,sub-048,totalimage,1,body,0,aortavois_ml-2_width-5_aorta_descending_lower,4,0.000745,124872000,560940.633000
3347984,sub-048,totalimage,1,body,0,aortavois_ml-2_width-5_aorta_descending_lower,5,0.000623,124872000,560940.633000
3347985,sub-048,totalimage,1,body,0,aortavois_ml-2_width-5_aorta_descending_lower,6,0.000621,124872000,560940.633000
3347986,sub-048,totalimage,1,body,0,aortavois_ml-2_width-5_aorta_descending_lower,7,0.000647,124872000,560940.633000
